In [14]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from etl import ETL
from feature_creation import FeatureCreation

plt.style.use("seaborn-v0_8")

In [87]:
df_yield = pd.read_csv("data/barley_yield_from_1982.csv", sep=";")  # 1982 to 2018
df_climate = pd.read_parquet(
    "data/climate_data_from_1982.parquet"
)  # 1982-2014 2015-2050
df_climate = df_climate[df_climate["year"] <= 2030]

In [88]:
df_yield, df_climate = ETL(df_yield, df_climate).run()

--- df_climate---
Departments/Scenario dropped because of any missing values:                                 department  scenario
date                                                
2015-01-01 12:00:00               Calvados  ssp2_4_5
2015-01-01 12:00:00            Deux_Sevres  ssp2_4_5
2015-01-01 12:00:00                Essonne  ssp2_4_5
2015-01-01 12:00:00                   Eure  ssp2_4_5
2015-01-01 12:00:00                  Rhone  ssp2_4_5
2015-01-01 12:00:00        Tarn_et_Garonne  ssp2_4_5
2015-01-01 12:00:00  Territoire_de_Belfort  ssp2_4_5
2015-01-01 12:00:00               Vaucluse  ssp2_4_5
--- df_yield ---
Departments dropped because of almost absolute absence of data:
 ['Hauts_de_Seine' 'Paris' 'Seine_SeineOise']


# Test of the module Feature Creation (feature_creation.py)


In [89]:
df_hist, df_forecast = FeatureCreation(df_yield, df_climate).run()

--- Amplitude feature created over phases ---
--- Threshold days feature created over phases ---
--- Temperature categories feature created ---
--- Consecutive days feature created ---


In [90]:
with pd.option_context("display.max_column", 999):
    display(df_hist.head())

,year,department,yield,area,production,scenario,code_dep,daily_NSA_temp_amplitude_Q1,daily_NSA_temp_amplitude_Q2,daily_NSA_temp_amplitude_Q3,daily_NSA_temp_amplitude_Q4,daily_NSA_temp_mean_Q1,daily_NSA_temp_mean_Q2,daily_NSA_temp_mean_Q3,daily_NSA_temp_mean_Q4,daily_NSA_temp_std_Q1,daily_NSA_temp_std_Q2,daily_NSA_temp_std_Q3,daily_NSA_temp_std_Q4,max_daily_NSA_temp_amplitude_Q1,max_daily_NSA_temp_amplitude_Q2,max_daily_NSA_temp_amplitude_Q3,max_daily_NSA_temp_amplitude_Q4,max_daily_NSA_temp_mean_Q1,max_daily_NSA_temp_mean_Q2,max_daily_NSA_temp_mean_Q3,max_daily_NSA_temp_mean_Q4,max_daily_NSA_temp_std_Q1,max_daily_NSA_temp_std_Q2,max_daily_NSA_temp_std_Q3,max_daily_NSA_temp_std_Q4,precipitation_amplitude_Q1,precipitation_amplitude_Q2,precipitation_amplitude_Q3,precipitation_amplitude_Q4,precipitation_mean_Q1,precipitation_mean_Q2,precipitation_mean_Q3,precipitation_mean_Q4,precipitation_std_Q1,precipitation_std_Q2,precipitation_std_Q3,precipitation_std_Q4,frost_days_germination,frost_days_growth,frost_days_maturation,heat_days_germination,heat_days_growth,heat_days_maturation,rainy_days_germination,rainy_days_growth,rainy_days_maturation,germination_below,germination_within,growth_above,growth_below,growth_within,maturation_above,maturation_below,maturation_within,max_consecutive_above_germination,max_consecutive_above_growth,max_consecutive_above_maturation,max_consecutive_below_germination,max_consecutive_below_growth,max_consecutive_below_maturation
0,1982,Ain,3.950080,16065.0,63458.0,historical,01,19.501831,19.293884,15.172821,23.353577,277.258392,284.543488,291.057281,278.157013,3.745729,4.552757,3.313892,5.635904,21.422180,23.057434,19.014343,27.926117,280.818390,289.230591,296.768921,282.986938,4.467213,5.475765,3.991581,6.313197,0.000566,0.000520,0.000501,0.000870,0.000109,0.000086,0.000043,0.000064,0.000119,0.000121,0.000092,0.000136,3.0,27.0,0.0,0.0,0.0,3.0,11.0,82.0,5.0,61.0,0.0,0.0,89.0,93.0,0.0,10.0,51.0,0.0,0.0,0.0,61.0,25.0,6.0
1,1983,Ain,2.648276,14500.0,38400.0,historical,01,18.292297,18.553345,16.015930,26.968567,275.036713,285.042694,289.147247,277.560028,4.230158,4.274082,3.196177,6.502086,18.219971,22.111542,19.338867,30.072388,278.627716,289.968994,294.333191,281.880524,4.289338,4.877942,3.892983,6.990046,0.000536,0.000509,0.000694,0.000416,0.000101,0.000070,0.000077,0.000069,0.000118,0.000102,0.000144,0.000104,5.0,48.0,0.0,0.0,0.0,0.0,22.0,62.0,14.0,61.0,0.0,0.0,102.0,80.0,0.0,24.0,37.0,0.0,0.0,0.0,61.0,46.0,12.0
2,1984,Ain,4.822580,15500.0,74750.0,historical,01,21.210449,19.159363,15.992889,21.758179,275.993958,284.692688,289.896545,277.596375,4.523038,4.058755,3.723015,4.483145,22.822693,23.874573,20.519470,22.541077,279.948517,289.600800,295.298187,281.683624,5.515414,4.871539,4.570998,5.316916,0.000420,0.000987,0.000635,0.000519,0.000076,0.000100,0.000052,0.000076,0.000108,0.000163,0.000107,0.000109,1.0,45.0,0.0,0.0,0.0,0.0,22.0,60.0,18.0,61.0,0.0,0.0,96.0,87.0,0.0,21.0,40.0,0.0,0.0,0.0,61.0,31.0,9.0
3,1985,Ain,4.196770,15500.0,65050.0,historical,01,17.951660,21.640381,8.846832,23.074890,274.739990,284.293854,290.097107,277.465149,4.352416,4.270841,2.101546,5.871760,22.054688,25.748901,11.690948,23.767120,278.670044,288.985718,295.426117,281.306885,5.321184,5.095441,3.149302,5.877611,0.000610,0.000538,0.000887,0.000813,0.000084,0.000094,0.000086,0.000110,0.000121,0.000121,0.000160,0.000172,6.0,48.0,0.0,0.0,0.0,0.0,25.0,66.0,17.0,61.0,0.0,0.0,99.0,83.0,0.0,22.0,39.0,0.0,0.0,0.0,61.0,32.0,11.0
4,1986,Ain,3.598450,12900.0,46420.0,historical,01,15.292114,21.908600,12.199310,20.551178,277.031097,284.711151,289.784210,278.067169,3.783030,5.290324,3.002419,4.504109,18.380554,26.323578,17.401245,22.073578,281.831177,289.722046,294.966492,282.639191,4.271338,6.182742,3.957856,5.580863,0.000266,0.000590,0.000724,0.000690,0.000047,0.000063,0.000071,0.000081,0.000074,0.000112,0.000127,0.000141,8.0,21.0,0.0,0.0,0.0,0.0,11.0,52.0,10.0,61.0,0.0,0.0,85.0,97.0,0.0,23.0,38.0,0.0,0.0,0.0,61.0,19.0,10.0


# Manual implementation of the feature creation module to test the functions


In [43]:
df_hist = df_yield
df_forecast = pd.DataFrame()

# Feature Creation


We will perform resample our df_climate dataset to set frequency $f$ that can be set to '1H' for hourly, '1D' for daily, '1W' for weekly, '1M' for monthly, '1Y' for yearly. We will then create new features based on the resampled data.


In [44]:
frequency = "1M"
group_col = ["scenario", "department", "code_dep", "year"]
metrics = df_climate.columns[~df_climate.columns.isin(group_col)]

## Amplitude


In [ ]:
def amplitude(group):
    return group.max() - group.min()


def mean(group):
    return group.mean()


dic_agg = {metric: [amplitude, mean] for metric in metrics}

In [52]:
monthly_data = (
    df_climate.groupby(by=group_col).resample(frequency).agg(dic_agg).reset_index()
)
monthly_data

scenario department code_dep  year       date max_daily_NSA_temp  \
                                                                amplitude   
0      historical        Ain       01  1982 1982-01-31           9.683777   
1      historical        Ain       01  1982 1982-02-28          14.983551   
2      historical        Ain       01  1982 1982-03-31          19.126007   
3      historical        Ain       01  1982 1982-04-30          19.172760   
4      historical        Ain       01  1982 1982-05-31          12.611084   
...           ...        ...      ...   ...        ...                ...   
84967    ssp5_8_5   Yvelines       78  2030 2030-08-31          12.264038   
84968    ssp5_8_5   Yvelines       78  2030 2030-09-30          10.770020   
84969    ssp5_8_5   Yvelines       78  2030 2030-10-31          12.176544   
84970    ssp5_8_5   Yvelines       78  2030 2030-11-30          10.665985   
84971    ssp5_8_5   Yvelines       78  2030 2030-12-31           8.901337   

                  daily_NSA_temp             precipitation            
             mean      amplitude        mean     amplitude      mean  
0      280.334229       8.102631  277.800964      0.000293  0.000142  
1      278.377472      12.605316  274.405182      0.000305  0.000056  
2      283.507233      15.886414  279.292877      0.000566  0.000123  
3      286.357605      14.585327  281.758759      0.000333  0.000082  
4      286.498291       8.893372  282.403473      0.000520  0.000154  
...           ...            ...         ...           ...       ...  
84967  301.677460       9.384827  295.994049      0.000054  0.000006  
84968  294.223724       7.198029  289.579651      0.000187  0.000024  
84969  287.950226      12.048645  284.209442      0.000410  0.000040  
84970  282.272888      10.144318  279.096252      0.000219  0.000027  
84971  280.958282       9.720154  278.849182      0.000188  0.000043  

[84972 rows x 11 columns]

In [ ]:
new_features = ["amp_" + feat for feat in metrics]
col_rename = {old_feat: new_feat for old_feat, new_feat in zip(metrics, new_features)}
monthly_data.rename(columns=col_rename, inplace=True)

In [53]:
monthly_data["date"] = monthly_data["date"].apply(lambda date: date.strftime("%b"))
monthly_data

scenario department code_dep  year date max_daily_NSA_temp  \
                                                          amplitude   
0      historical        Ain       01  1982  Jan           9.683777   
1      historical        Ain       01  1982  Feb          14.983551   
2      historical        Ain       01  1982  Mar          19.126007   
3      historical        Ain       01  1982  Apr          19.172760   
4      historical        Ain       01  1982  May          12.611084   
...           ...        ...      ...   ...  ...                ...   
84967    ssp5_8_5   Yvelines       78  2030  Aug          12.264038   
84968    ssp5_8_5   Yvelines       78  2030  Sep          10.770020   
84969    ssp5_8_5   Yvelines       78  2030  Oct          12.176544   
84970    ssp5_8_5   Yvelines       78  2030  Nov          10.665985   
84971    ssp5_8_5   Yvelines       78  2030  Dec           8.901337   

                  daily_NSA_temp             precipitation            
             mean      amplitude        mean     amplitude      mean  
0      280.334229       8.102631  277.800964      0.000293  0.000142  
1      278.377472      12.605316  274.405182      0.000305  0.000056  
2      283.507233      15.886414  279.292877      0.000566  0.000123  
3      286.357605      14.585327  281.758759      0.000333  0.000082  
4      286.498291       8.893372  282.403473      0.000520  0.000154  
...           ...            ...         ...           ...       ...  
84967  301.677460       9.384827  295.994049      0.000054  0.000006  
84968  294.223724       7.198029  289.579651      0.000187  0.000024  
84969  287.950226      12.048645  284.209442      0.000410  0.000040  
84970  282.272888      10.144318  279.096252      0.000219  0.000027  
84971  280.958282       9.720154  278.849182      0.000188  0.000043  

[84972 rows x 11 columns]

In [289]:
pivoted_df = monthly_data.pivot_table(
    index=["scenario", "nom_dep", "code_dep", "year"],
    columns="date",
    values=new_features,
).reset_index()
pivoted_df.columns = [
    f"{metric}_{month}".strip("_") for metric, month in pivoted_df.columns
]
pivoted_df.set_index(["year"], drop=True, inplace=True)
pivoted_df.index = pd.to_datetime(pivoted_df.index, format="%Y")
pivoted_df

,scenario,nom_dep,code_dep,amp_daily_NSA_temp_Apr,amp_daily_NSA_temp_Aug,amp_daily_NSA_temp_Dec,amp_daily_NSA_temp_Feb,amp_daily_NSA_temp_Jan,amp_daily_NSA_temp_Jul,amp_daily_NSA_temp_Jun,...,amp_precipitation_Dec,amp_precipitation_Feb,amp_precipitation_Jan,amp_precipitation_Jul,amp_precipitation_Jun,amp_precipitation_Mar,amp_precipitation_May,amp_precipitation_Nov,amp_precipitation_Oct,amp_precipitation_Sep
year,,,,,,,,,,,,,,,,,,,,,
1982-01-01,historical,Ain,01,14.585327,13.266602,14.508118,12.605316,8.102631,8.545471,9.951599,...,0.000337,0.000305,0.000293,0.000355,0.000220,0.000566,0.000520,0.000488,0.000870,0.000160
1983-01-01,historical,Ain,01,11.894348,7.150391,16.380951,17.021851,9.464539,10.034546,10.661621,...,0.000416,0.000397,0.000439,0.000694,0.000392,0.000536,0.000355,0.000363,0.000217,0.000531
1984-01-01,historical,Ain,01,14.128510,5.867737,12.289917,9.180328,15.847443,10.030670,9.929626,...,0.000269,0.000420,0.000419,0.000635,0.000335,0.000334,0.000682,0.000384,0.000519,0.000394
1985-01-01,historical,Ain,01,11.441620,8.075775,14.267517,13.399414,14.638794,6.790100,11.491058,...,0.000385,0.000473,0.000610,0.000887,0.000538,0.000358,0.000422,0.000802,0.000813,0.000430
1986-01-01,historical,Ain,01,11.271729,10.372467,9.982452,11.505005,10.697601,9.380066,12.694794,...,0.000542,0.000221,0.000214,0.000724,0.000288,0.000266,0.000588,0.000690,0.000190,0.000420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046-01-01,ssp5_8_5,Yvelines,78,10.671600,11.138885,10.903961,10.098511,6.579498,10.647217,13.361725,...,0.000108,0.000067,0.000138,0.000076,0.000130,0.000076,0.000143,0.000382,0.000298,0.000022
2047-01-01,ssp5_8_5,Yvelines,78,5.617401,10.944977,10.930328,12.897888,16.145782,16.649200,14.308533,...,0.000135,0.000169,0.000239,0.000226,0.000100,0.000119,0.000227,0.000189,0.000077,0.000192
2048-01-01,ssp5_8_5,Yvelines,78,8.661102,8.688995,10.149933,13.638367,9.829681,13.737396,11.352478,...,0.000118,0.000153,0.000218,0.000495,0.000294,0.000374,0.000173,0.000214,0.000326,0.000256


Merge to df_yield on year and department.


In [290]:
df_hist = pd.merge(
    df_hist,
    pivoted_df[pivoted_df["scenario"] == "historical"],
    left_on=["department", "year"],
    right_on=["nom_dep", "year"],
    how="outer",
)

In [291]:
df_forecast = pivoted_df[~(pivoted_df["scenario"] == "historical")]

In [292]:
df_hist.head()

,department,yield,area,production,scenario,nom_dep,code_dep,amp_daily_NSA_temp_Apr,amp_daily_NSA_temp_Aug,amp_daily_NSA_temp_Dec,...,amp_precipitation_Dec,amp_precipitation_Feb,amp_precipitation_Jan,amp_precipitation_Jul,amp_precipitation_Jun,amp_precipitation_Mar,amp_precipitation_May,amp_precipitation_Nov,amp_precipitation_Oct,amp_precipitation_Sep
year,,,,,,,,,,,,,,,,,,,,,
1982-01-01,Ain,3.950080,16065.0,63458.0,historical,Ain,01,14.585327,13.266602,14.508118,...,0.000337,0.000305,0.000293,0.000355,0.000220,0.000566,0.000520,0.000488,0.000870,0.000160
1983-01-01,Ain,2.648276,14500.0,38400.0,historical,Ain,01,11.894348,7.150391,16.380951,...,0.000416,0.000397,0.000439,0.000694,0.000392,0.000536,0.000355,0.000363,0.000217,0.000531
1984-01-01,Ain,4.822580,15500.0,74750.0,historical,Ain,01,14.128510,5.867737,12.289917,...,0.000269,0.000420,0.000419,0.000635,0.000335,0.000334,0.000682,0.000384,0.000519,0.000394
1985-01-01,Ain,4.196770,15500.0,65050.0,historical,Ain,01,11.441620,8.075775,14.267517,...,0.000385,0.000473,0.000610,0.000887,0.000538,0.000358,0.000422,0.000802,0.000813,0.000430
1986-01-01,Ain,3.598450,12900.0,46420.0,historical,Ain,01,11.271729,10.372467,9.982452,...,0.000542,0.000221,0.000214,0.000724,0.000288,0.000266,0.000588,0.000690,0.000190,0.000420


# Threshold Features


In [86]:
thresholds = {"rain": 8.493e-05, "frost": 0 + 273.15, "heat": 30 + 273.15}


def rainy_days(group):
    return (group > thresholds["rain"]).sum()


def frost_days(group):
    return (group < thresholds["frost"]).sum()


def heat_days(group):
    return (group > thresholds["heat"]).sum()


rain_col = "precipitation"
max_temp_col = "max_daily_NSA_temp"
mean_temp_col = "daily_NSA_temp"

dic_agg = {rain_col: rainy_days, max_temp_col: heat_days, mean_temp_col: frost_days}

monthly_data = (
    df_climate.groupby(by=group_col).resample(frequency).agg(dic_agg).reset_index()
)

In [296]:
rename_dic = {col: func.__name__ for col, func in dic_agg.items()}
monthly_data.rename(columns=rename_dic, inplace=True)

monthly_data["date"] = monthly_data["date"].apply(lambda date: date.strftime("%b"))

monthly_data

,scenario,nom_dep,code_dep,year,date,rainy_days,heat_days,frost_days
0,historical,Ain,01,1982,Jan,21,0,0
1,historical,Ain,01,1982,Feb,8,0,10
2,historical,Ain,01,1982,Mar,17,0,2
3,historical,Ain,01,1982,Apr,13,0,0
4,historical,Ain,01,1982,May,16,0,0
...,...,...,...,...,...,...,...,...
147127,ssp5_8_5,Yvelines,78,2050,Aug,4,0,0
147128,ssp5_8_5,Yvelines,78,2050,Sep,3,0,0
147129,ssp5_8_5,Yvelines,78,2050,Oct,2,0,0
147130,ssp5_8_5,Yvelines,78,2050,Nov,3,0,0


In [305]:
# Pivot from monthly data to yearly data
pivoted_df = monthly_data.pivot_table(
    index=["scenario", "nom_dep", "code_dep", "year"],
    columns="date",
    values=list(rename_dic.values()),
).reset_index()

pivoted_df.columns = [
    f"{metric}_{month}".strip("_") for metric, month in pivoted_df.columns
]

In [306]:
pivoted_df.set_index(["year"], drop=True, inplace=True)
pivoted_df.index = pd.to_datetime(pivoted_df.index, format="%Y")
pivoted_df

,scenario,nom_dep,code_dep,frost_days_Apr,frost_days_Aug,frost_days_Dec,frost_days_Feb,frost_days_Jan,frost_days_Jul,frost_days_Jun,...,rainy_days_Dec,rainy_days_Feb,rainy_days_Jan,rainy_days_Jul,rainy_days_Jun,rainy_days_Mar,rainy_days_May,rainy_days_Nov,rainy_days_Oct,rainy_days_Sep
year,,,,,,,,,,,,,,,,,,,,,
1982-01-01,historical,Ain,01,0.0,0.0,15.0,10.0,0.0,0.0,0.0,...,7.0,8.0,21.0,3.0,2.0,17.0,16.0,6.0,5.0,2.0
1983-01-01,historical,Ain,01,0.0,0.0,21.0,11.0,11.0,0.0,0.0,...,7.0,11.0,14.0,6.0,8.0,13.0,10.0,18.0,4.0,9.0
1984-01-01,historical,Ain,01,1.0,0.0,15.0,10.0,19.0,0.0,0.0,...,8.0,8.0,14.0,7.0,11.0,7.0,13.0,11.0,11.0,5.0
1985-01-01,historical,Ain,01,1.0,0.0,15.0,19.0,13.0,0.0,0.0,...,9.0,13.0,9.0,12.0,5.0,5.0,18.0,8.0,17.0,2.0
1986-01-01,historical,Ain,01,0.0,0.0,8.0,2.0,11.0,0.0,0.0,...,17.0,2.0,7.0,7.0,3.0,10.0,10.0,8.0,3.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046-01-01,ssp5_8_5,Yvelines,78,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,2.0,0.0,3.0,0.0,1.0,0.0,3.0,5.0,3.0,0.0
2047-01-01,ssp5_8_5,Yvelines,78,0.0,0.0,1.0,3.0,8.0,0.0,0.0,...,1.0,3.0,1.0,1.0,1.0,2.0,1.0,4.0,0.0,4.0
2048-01-01,ssp5_8_5,Yvelines,78,0.0,0.0,3.0,9.0,6.0,0.0,0.0,...,2.0,5.0,2.0,2.0,4.0,2.0,2.0,6.0,3.0,2.0


In [307]:
pd.merge(
    df_hist,
    pivoted_df[pivoted_df["scenario"] == "historical"],
    left_on=["nom_dep", "year", "code_dep", "scenario"],
    right_on=["nom_dep", "year", "code_dep", "scenario"],
    how="outer",
)

,department,yield,area,production,scenario,nom_dep,code_dep,amp_daily_NSA_temp_Apr,amp_daily_NSA_temp_Aug,amp_daily_NSA_temp_Dec,...,rainy_days_Dec,rainy_days_Feb,rainy_days_Jan,rainy_days_Jul,rainy_days_Jun,rainy_days_Mar,rainy_days_May,rainy_days_Nov,rainy_days_Oct,rainy_days_Sep
year,,,,,,,,,,,,,,,,,,,,,
1982-01-01,Ain,3.950080,16065.0,63458.0,historical,Ain,01,14.585327,13.266602,14.508118,...,7.0,8.0,21.0,3.0,2.0,17.0,16.0,6.0,5.0,2.0
1983-01-01,Ain,2.648276,14500.0,38400.0,historical,Ain,01,11.894348,7.150391,16.380951,...,7.0,11.0,14.0,6.0,8.0,13.0,10.0,18.0,4.0,9.0
1984-01-01,Ain,4.822580,15500.0,74750.0,historical,Ain,01,14.128510,5.867737,12.289917,...,8.0,8.0,14.0,7.0,11.0,7.0,13.0,11.0,11.0,5.0
1985-01-01,Ain,4.196770,15500.0,65050.0,historical,Ain,01,11.441620,8.075775,14.267517,...,9.0,13.0,9.0,12.0,5.0,5.0,18.0,8.0,17.0,2.0
1986-01-01,Ain,3.598450,12900.0,46420.0,historical,Ain,01,11.271729,10.372467,9.982452,...,17.0,2.0,7.0,7.0,3.0,10.0,10.0,8.0,3.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-01-01,Yvelines,7.037840,9624.0,67732.2,historical,Yvelines,78,9.208282,10.101349,10.645935,...,2.0,2.0,3.0,4.0,0.0,1.0,1.0,5.0,2.0,1.0
2011-01-01,Yvelines,6.155870,8766.0,53962.4,historical,Yvelines,78,9.366638,13.055542,8.522217,...,3.0,1.0,1.0,1.0,4.0,5.0,8.0,3.0,6.0,1.0
2012-01-01,Yvelines,7.675490,9100.0,69847.0,historical,Yvelines,78,11.627319,11.498718,12.754639,...,2.0,1.0,2.0,0.0,0.0,2.0,3.0,5.0,5.0,2.0


In [309]:
pd.merge(
    df_forecast,
    pivoted_df[~(pivoted_df["scenario"] == "historical")],
    on=["nom_dep", "year", "scenario", "code_dep"],
    how="outer",
)

,scenario,nom_dep,code_dep,amp_daily_NSA_temp_Apr,amp_daily_NSA_temp_Aug,amp_daily_NSA_temp_Dec,amp_daily_NSA_temp_Feb,amp_daily_NSA_temp_Jan,amp_daily_NSA_temp_Jul,amp_daily_NSA_temp_Jun,...,rainy_days_Dec,rainy_days_Feb,rainy_days_Jan,rainy_days_Jul,rainy_days_Jun,rainy_days_Mar,rainy_days_May,rainy_days_Nov,rainy_days_Oct,rainy_days_Sep
year,,,,,,,,,,,,,,,,,,,,,
2015-01-01,ssp1_2_6,Ain,01,13.302734,10.143005,15.577698,6.312073,5.736542,11.332397,9.489990,...,7.0,8.0,10.0,6.0,3.0,12.0,8.0,9.0,4.0,11.0
2016-01-01,ssp1_2_6,Ain,01,12.063354,11.057465,9.628601,11.505615,8.315643,8.163452,8.313171,...,9.0,9.0,12.0,11.0,5.0,5.0,13.0,14.0,15.0,6.0
2017-01-01,ssp1_2_6,Ain,01,8.806732,10.507019,12.481995,11.021942,13.921326,6.458191,10.793091,...,11.0,9.0,11.0,6.0,5.0,8.0,6.0,12.0,10.0,5.0
2018-01-01,ssp1_2_6,Ain,01,14.717407,8.809509,13.714783,6.127136,10.296631,11.391785,8.336487,...,11.0,7.0,9.0,8.0,8.0,7.0,18.0,18.0,17.0,6.0
2019-01-01,ssp1_2_6,Ain,01,16.814819,8.586914,9.677094,12.183472,8.142578,14.037292,10.136383,...,11.0,5.0,6.0,9.0,11.0,5.0,8.0,15.0,13.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046-01-01,ssp5_8_5,Yvelines,78,10.671600,11.138885,10.903961,10.098511,6.579498,10.647217,13.361725,...,2.0,0.0,3.0,0.0,1.0,0.0,3.0,5.0,3.0,0.0
2047-01-01,ssp5_8_5,Yvelines,78,5.617401,10.944977,10.930328,12.897888,16.145782,16.649200,14.308533,...,1.0,3.0,1.0,1.0,1.0,2.0,1.0,4.0,0.0,4.0
2048-01-01,ssp5_8_5,Yvelines,78,8.661102,8.688995,10.149933,13.638367,9.829681,13.737396,11.352478,...,2.0,5.0,2.0,2.0,4.0,2.0,2.0,6.0,3.0,2.0


# Lagged Features


In [27]:
df_hist

,year,yield,area,production,scenario,nom_dep,code_dep,amp_daily_NSA_temp_1,amp_daily_NSA_temp_2,amp_daily_NSA_temp_3,...,max_consecutive_above_maturation,max_consecutive_below_germination,max_consecutive_below_growth,max_consecutive_below_maturation,max_consecutive_within_germination,max_consecutive_within_growth,max_consecutive_within_maturation,annual_precip_below,annual_precip_within,annual_precip_above
0,1982,3.950080,16065.0,63458.0,historical,Ain,01,8.102631,12.605316,15.886414,...,0.0,61.0,25.0,6.0,0.0,27.0,41.0,0.0,0.0,1.0
1,1983,2.648276,14500.0,38400.0,historical,Ain,01,9.464539,17.021851,11.039673,...,0.0,61.0,46.0,12.0,0.0,48.0,16.0,0.0,0.0,1.0
2,1984,4.822580,15500.0,74750.0,historical,Ain,01,15.847443,9.180328,7.104919,...,0.0,61.0,31.0,9.0,0.0,34.0,16.0,0.0,0.0,1.0
3,1985,4.196770,15500.0,65050.0,historical,Ain,01,14.638794,13.399414,9.194336,...,0.0,61.0,32.0,11.0,0.0,39.0,16.0,0.0,0.0,1.0
4,1986,3.598450,12900.0,46420.0,historical,Ain,01,10.697601,11.505005,9.750580,...,0.0,61.0,19.0,10.0,0.0,31.0,24.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3473,2010,7.037840,9624.0,67732.2,historical,Yvelines,78,9.430847,9.205994,8.768127,...,9.0,61.0,9.0,2.0,0.0,37.0,11.0,0.0,0.0,1.0
3474,2011,6.155870,8766.0,53962.4,historical,Yvelines,78,13.514313,10.069000,9.415466,...,2.0,61.0,21.0,2.0,0.0,59.0,23.0,0.0,0.0,1.0
3475,2012,7.675490,9100.0,69847.0,historical,Yvelines,78,16.795593,10.042389,9.088135,...,5.0,61.0,16.0,6.0,0.0,58.0,26.0,0.0,0.0,1.0
3476,2013,7.043820,10360.0,72974.0,historical,Yvelines,78,11.944244,10.872925,14.288330,...,2.0,61.0,14.0,1.0,0.0,55.0,22.0,0.0,0.0,1.0
